<a href="https://colab.research.google.com/github/0xJCarlos/QuestionAnswering/blob/main/QuestionAnswering_Esp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Primeramente instalamos las librerias necesarias

In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
!pip install transformers datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.8 MB/s eta 0:00:00


In [3]:
## Cargar dataset
from datasets import load_dataset
squad = load_dataset("squad_es", 'v2.0.0', split="train[:5000]")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [4]:
print(squad.shape)

(5000, 5)


In [5]:
## Dividir el dataset en sets de train y test

squad = squad.train_test_split(test_size=0.2)

In [6]:
## Observar un ejemplo
squad["train"][0]

{'id': '56d5ecf21c85041400946e5c',
 'title': '2008: Terremoto',
 'context': 'Las fuertes lluvias persistentes y los deslizamientos de tierra en el condado de Wenchuan y la zona cercana afectaron gravemente los esfuerzos de rescate. Al inicio de las operaciones de rescate el 12 de mayo, se desplegaron 20 helicópteros para la entrega de alimentos, agua y ayuda de emergencia, y también la evacuación de los heridos y el reconocimiento de las áreas afectadas por el cuádruple. A las 17: 37 CST del 13 de mayo, un total de más de 15.600 soldados y milicianos reservistas de la Región Militar de Chengdu se habían unido a la fuerza de rescate en las zonas muy afectadas. Un comandante informó de la ciudad de Yingxiu, Wenchuan, que se encontraron alrededor de 3.000 sobrevivientes, mientras que el estado de los otros habitantes (alrededor de 9.000) permaneció poco claro. Los 1.300 rescatadores llegaron al epicentro, y 300 tropas pioneras alcanzaron la sede de Wenchuan a las 23: 30 CST. A las 12: 17 

In [7]:
## Cargar DistilBERT tokenizer para procesar los campos "question" y "context"
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("dccuchile/distilbert-base-spanish-uncased-finetuned-qa-mlqa")

tokenizer_config.json:   0%|          | 0.00/397 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/486k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [8]:
### HAYALGUNOS PASOS DE PREPROCESAMIENTO PARA QUESTION-ANSWERING DE LOS QUE SE DEBERÍA ESTÁR CONSCIENTES:

#1. Some examples in a dataset may have a very long context that exceeds the maximum input length of the model.
#To deal with longer sequences, truncate only the context by setting truncation="only_second"

#2. Next, map the start and end positions of the answer to the original context by setting
# return_offset_mapping=True

#3. With the mapping in hand, now you can find the start and end tokens of the answer.
#Use the sequence_ids method to find which part of the offset corresponds to the question and which corresponds to the context.

In [9]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    contexts = [c.strip() for c in examples["context"]]

    inputs = tokenizer(
        questions,
        contexts,
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, (offset, answer) in enumerate(zip(offset_mapping, answers)):
        # Check if answer exists
        if len(answer["answer_start"]) > 0 and len(answer["text"]) > 0:
            start_char = answer["answer_start"][0]
            end_char = answer["answer_start"][0] + len(answer["text"][0])
        else:
            start_char = 0
            end_char = 0

        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs


In [10]:
## Usar la funcion map para aplicar la función de preprocesamiento en todo el dataset. Puedes acelerarlo usando batched=True
tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [11]:
### Crear un batch de ejemplos usando DefaultDataCollator. No aplica ningún preprocesamiento adicional como padding
from transformers import DefaultDataCollator
data_collator = DefaultDataCollator(return_tensors="tf")

## Entrenar el modelo!

In [12]:
from transformers import  create_optimizer

batch_size = 16
num_epochs = 2
total_train_steps = (len(tokenized_squad["train"]) // batch_size) * num_epochs
optimizer, schedule = create_optimizer(
    init_lr = 2e-5,
    num_warmup_steps = 0,
    num_train_steps = total_train_steps,
)

In [13]:
from transformers import TFAutoModelForQuestionAnswering

model = TFAutoModelForQuestionAnswering.from_pretrained("dccuchile/distilbert-base-spanish-uncased-finetuned-qa-mlqa", from_pt=True)

config.json:   0%|          | 0.00/575 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/267M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFDistilBertForQuestionAnswering.

All the weights of TFDistilBertForQuestionAnswering were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForQuestionAnswering for predictions without further training.


In [14]:
#Convertir dataset a formato tf.data.Dataset con prepare_tf_dataset():
tf_train_set = model.prepare_tf_dataset(
    tokenized_squad["train"],
    shuffle = True,
    batch_size = 16,
    collate_fn = data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_squad["test"],
    shuffle = False,
    batch_size = 16,
    collate_fn = data_collator,
)

In [15]:
#Configura el modelo para entrenamiento con compile
import tensorflow as tf
model.compile(optimizer=optimizer)

In [16]:
from transformers.keras_callbacks import PushToHubCallback

callback = PushToHubCallback(
    output_dir="QuestionAnswer_ESP",
    tokenizer=tokenizer,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/0xJCarlos/QuestionAnswer_ESP into local empty directory.


Download file tf_model.h5:   0%|          | 25.8k/255M [00:00<?, ?B/s]

Clean file tf_model.h5:   0%|          | 1.00k/255M [00:00<?, ?B/s]

In [17]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=5, callbacks=[callback])

Epoch 1/5
250/250 [==============================] - 238s 868ms/step - loss: 1.9224 - val_loss: 1.7523
Epoch 2/5
250/250 [==============================] - 221s 882ms/step - loss: 1.4224 - val_loss: 1.7276
Epoch 3/5
250/250 [==============================] - 197s 787ms/step - loss: 1.3102 - val_loss: 1.7276
Epoch 4/5
250/250 [==============================] - 196s 783ms/step - loss: 1.3096 - val_loss: 1.7276
Epoch 5/5
250/250 [==============================] - 199s 798ms/step - loss: 1.3181 - val_loss: 1.7276


In [19]:
model.save_weights('/content/drive/MyDrive/Servicio Social/QuestionAnswer_ESP_5Epochs.h5')

In [20]:
question = "¿Cual fue su ultimo club?"
context = "David Michael Bentley (n. Peterborough, Reino Unido, el 27 de agosto de 1984) es un exfutbolista inglés que jugaba como centrocampista y su último club fue el Tottenham Hotspurs. Jugó en la selección de fútbol de Inglaterra. "

In [21]:
from transformers import pipeline

question_answerer = pipeline("question-answering", model="QuestionAnswer_ESP")

question_answerer(question=question, context=context)

Some layers from the model checkpoint at QuestionAnswer_ESP were not used when initializing TFDistilBertForQuestionAnswering: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForQuestionAnswering were not initialized from the model checkpoint at QuestionAnswer_ESP and are newly initialized: ['dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'score': 0.7573922872543335,
 'start': 159,
 'end': 177,
 'answer': 'Tottenham Hotspurs'}